In [1]:
try:
  import sys
  sys.path.append('../')
  from modules.validate_prop import *
except ImportError:
  url = 'https://raw.githubusercontent.com/jhjeong314/Proofmood/main/modules'
  import httpimport
  with httpimport.remote_repo(url):
    from validate_prop import *

from pprint import pprint

In [2]:
def show_formula(input_text):
  from IPython.display import display, Math
  try:
    ast = parse_ast(input_text)
  except ValueError as e:
    print(e)
  
  s = ast.build_infix('latex') # type: ignore
  display(Math(f"${s}$"))

In [3]:
# repeat 
fmla = 'A imp A'
show_formula(fmla)

prf_str0 = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. not A .hyp
    proves
    4. bot .bot intro 2,3
  5. A .not elim 3-4
6. A imp A .imp intro 2-5
'''
proof = parse_fitch(prf_str0)
proof.show_fitch_text()

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. not A	 .hyp
││├─
│││4. bot	✓ bot intro 2,3
││5. A	✓ not elim 3-4
│6. A imp A	✓ imp intro 2-5


In [4]:
# error case 1
# annotation is syntactically correct 
# but the proof validation fails
# red x marks the error

fmla = 'A imp A'
show_formula(fmla)

prf_str0 = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. not A .hyp
    proves
    4. bot .bot intro 2,23
  5. A .not elim 3-4
6. A imp A .imp intro 2-5
'''
proof = parse_fitch(prf_str0)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

is_earlier(): node_code=23 not found in index_dict

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. not A	 .hyp
││├─
│││4. bot	x bot intro 2,23
││5. A	✓ not elim 3-4
│6. A imp A	✓ imp intro 2-5


In [5]:
# error case 2
# annotation is syntactically incorrect
# the whole annotation appears in red color

fmla = 'A imp A'
show_formula(fmla)

prf_str0 = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. not A .hyp
    proves
    4. bot .bot intro 2
  5. A .not elim 3-4
6. A imp A .imp intro 2-5
'''
proof = parse_fitch(prf_str0)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. not A	 .hyp
││├─
│││4. bot	x bot intro 2
││5. A	✓ not elim 3-4
│6. A imp A	✓ imp intro 2-5


In [6]:
# error case 3
# invalid proof tree structure
# try-except block catches the error to hide the lengthy traceback

fmla = 'A imp A'
show_formula(fmla)

prf_str0 = '''
1. .hyp
proves
  2. A .hyp
    3. not A .hyp
    proves
    4. bot .bot intro 2
  5. A .not elim 3-4
6. A imp A .imp intro 2-5
'''
try:
  proof = parse_fitch(prf_str0)
  proof.show_fitch_text(1)
except Exception as e:
  print("Proof tree structure error:\n")
  print(e)


<IPython.core.display.Math object>

Proof tree structure error:

get_str_li(): level increased while belongs_to_hyp
	3. not A .hyp


In [7]:
# LEM(Law of Excluded Middle)
# A or not A
fmla = 'A or not A'
show_formula(fmla)

prf_str1 = '''
1. .hyp
proves
  # comment line takes up a line
  3. not(A or not A) .hyp
  proves
    4. A .hyp
    proves
    5. A or not A .or intro 4
    6. bot .bot intro 5,3
  7. not A .not intro 4-6
  8. A or not A .or intro 7
  9. bot .bot intro 8,3
10. A or not A .not elim 2-9
#^ try replacing 2-9 with 3-9 and see what happens
'''
proof = parse_fitch(prf_str1)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. # comment line takes up a line
││3. not (A or not A)	 .hyp
│├─
│││4. A	 .hyp
││├─
│││5. A or not A	✓ or intro 4
│││6. bot	✓ bot intro 5,3
││7. not A	✓ not intro 4-6
││8. A or not A	✓ or intro 7
││9. bot	✓ bot intro 8,3
│10. A or not A	✓ not elim 2-9
│11. #^ try replacing 2-9 with 3-9 and see what happens


In [8]:
# Axiom 1 (Hilbert, Lukasiewicz)
fmla = 'A imp (B imp A)'
show_formula(fmla)

prf_str2 = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. B .hyp
    proves
    4. A .repeat 2
  5. B imp A .imp intro 3-4
6. A imp (B imp A) .imp intro 2-5
'''
proof = parse_fitch(prf_str2)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. B	 .hyp
││├─
│││4. A	✓ repeat 2
││5. B imp A	✓ imp intro 3-4
│6. A imp B imp A	✓ imp intro 2-5


In [9]:
# Axiom 2 (Hilbert, Lukasiewicz)
fmla = '(A imp (B imp C)) imp ((A imp B) imp (A imp C))'
show_formula(fmla)

prf_str3 = '''
1. .hyp
proves
  2. A imp (B imp C) .hyp
  proves
    3. A imp B .hyp
    proves
      4. A .hyp
      proves
      5. B .imp elim 3,4
      6. B imp C .imp elim 2,4
      7. C .imp elim 6,5
    8. A imp C .imp intro 4-7
  9. (A imp B) imp (A imp C) .imp intro 3-8
10. (A imp (B imp C)) imp ((A imp B) imp (A imp C)) .imp intro 2-9
'''
proof = parse_fitch(prf_str3)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A imp B imp C	 .hyp
│├─
│││3. A imp B	 .hyp
││├─
││││4. A	 .hyp
│││├─
││││5. B	✓ imp elim 3,4
││││6. B imp C	✓ imp elim 2,4
││││7. C	✓ imp elim 6,5
│││8. A imp C	✓ imp intro 4-7
││9. (A imp B) imp A imp C	✓ imp intro 3-8
│10. (A imp B imp C) imp (A imp B) imp A imp C	✓ imp intro 2-9


In [10]:
# Axiom 3 (Hilbert, Lukasiewicz), aka contraposition
fmla = '(not B imp not A) imp (A imp B)'
show_formula(fmla)

prf_str4 = '''
1. .hyp
proves
  2. not B imp not A .hyp
  proves
    3. A .hyp
    proves
      4. not B .hyp
      proves
      5. not A .imp elim 2,4
      6. bot .bot intro 3,5
    7. B .not elim 4-6
  8. A imp B .imp intro 3-7
9. (not B imp not A) imp (A imp B) .imp intro 2-8
'''
proof = parse_fitch(prf_str4)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. not B imp not A	 .hyp
│├─
│││3. A	 .hyp
││├─
││││4. not B	 .hyp
│││├─
││││5. not A	✓ imp elim 2,4
││││6. bot	✓ bot intro 3,5
│││7. B	✓ not elim 4-6
││8. A imp B	✓ imp intro 3-7
│9. (not B imp not A) imp A imp B	✓ imp intro 2-8


In [11]:
# not A imp (A imp B)
fmla = 'not A imp (A imp B)'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. not A .hyp
  proves
    3. A .hyp
    proves
    4. bot .bot intro 2,3
    5. B .bot elim 4
  6. A imp B .imp intro 3-5
7. not A imp (A imp B) .imp intro 2-6
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. not A	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. bot	✓ bot intro 2,3
│││5. B	✓ bot elim 4
││6. A imp B	✓ imp intro 3-5
│7. not A imp A imp B	✓ imp intro 2-6


In [12]:
fmla = 'A imp ((A imp B) imp B)'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. A imp B .hyp
    proves
      4. A .hyp
      proves
      5. B .imp elim 3,4
    6. A imp B .imp intro 4-5
    7. B .imp elim 6,2
  8. (A imp B) imp B .imp intro 3-7
9. A imp ((A imp B) imp B) .imp intro 2-8
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. A imp B	 .hyp
││├─
││││4. A	 .hyp
│││├─
││││5. B	✓ imp elim 3,4
│││6. A imp B	✓ imp intro 4-5
│││7. B	✓ imp elim 6,2
││8. (A imp B) imp B	✓ imp intro 3-7
│9. A imp (A imp B) imp B	✓ imp intro 2-8


In [13]:
# Transitivity of Implication
fmla = '(A imp B) imp ((B imp C) imp (A imp C))'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. A imp B .hyp
  proves
    3. B imp C .hyp
    proves
      4. A .hyp
      proves
      5. B .imp elim 2,4
      6. C .imp elim 3,5
    7. A imp C .imp intro 4-6
  8. (B imp C) imp (A imp C) .imp intro 3-7
9. (A imp B) imp ((B imp C) imp (A imp C)) .imp intro 2-8
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A imp B	 .hyp
│├─
│││3. B imp C	 .hyp
││├─
││││4. A	 .hyp
│││├─
││││5. B	✓ imp elim 2,4
││││6. C	✓ imp elim 3,5
│││7. A imp C	✓ imp intro 4-6
││8. (B imp C) imp A imp C	✓ imp intro 3-7
│9. (A imp B) imp (B imp C) imp A imp C	✓ imp intro 2-8


In [14]:
# Transitivity of implication may be easier to understand
# if we write it as follows:
#   (A imp B) and (B imp C) imp (A imp C)
# In fact, the following is a tautology.
fmla = '(A imp (B imp C)) iff (A and B imp C)'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. A imp (B imp C) .hyp
  proves
    3. A and B .hyp
    proves
    4. A .and elim 3
    5. B imp C .imp elim 2,4
    6. B .and elim 3
    7. C .imp elim 5,6
  8. A and B imp C .imp intro 3-7
  9. A and B imp C .hyp
  proves
    10. A .hyp
    proves
      11. B  .hyp
      proves
      12. A and B .and intro 10,11
      13. C .imp elim 9,12
    14. B imp C .imp intro 11-13
  15. A imp (B imp C) .imp intro 10-14
16. (A and B imp C) iff (A imp (B imp C)) .iff intro 2-8, 9-15
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)


<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A imp B imp C	 .hyp
│├─
│││3. A and B	 .hyp
││├─
│││4. A	✓ and elim 3
│││5. B imp C	✓ imp elim 2,4
│││6. B	✓ and elim 3
│││7. C	✓ imp elim 5,6
││8. A and B imp C	✓ imp intro 3-7
││9. A and B imp C	 .hyp
│├─
│││10. A	 .hyp
││├─
││││11. B	 .hyp
│││├─
││││12. A and B	✓ and intro 10,11
││││13. C	✓ imp elim 9,12
│││14. B imp C	✓ imp intro 11-13
││15. A imp B imp C	✓ imp intro 10-14
│16. (A and B imp C) iff (A imp B imp C)	✓ iff intro 2-8,9-15


In [15]:
# So, the following is a tautology.
# (A imp B imp C) iff (B imp A imp C).

In [16]:
# Double negation elimination
fmla = 'not not A imp A'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. not not A .hyp
  proves
    3. not A .hyp
    proves
    4. bot .bot intro 2,3
  5. A .not elim 3-4
7. not not A imp A .imp intro 2-5
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. not not A	 .hyp
│├─
│││3. not A	 .hyp
││├─
│││4. bot	✓ bot intro 2,3
││5. A	✓ not elim 3-4
│6. not not A imp A	✓ imp intro 2-5


In [17]:
# Double negation introduction
fmla = 'A imp not not A'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. A .hyp
  proves
    3. not A .hyp
    proves
    4. bot .bot intro 2,3
  5. not not A .not intro 3-4
7. A imp not not A .imp intro 2-5
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(1)

<IPython.core.display.Math object>

│1. 	 .hyp
├─
││2. A	 .hyp
│├─
│││3. not A	 .hyp
││├─
│││4. bot	✓ bot intro 2,3
││5. not not A	✓ not intro 3-4
│6. A imp not not A	✓ imp intro 2-5


In [18]:
# Following is a version of the Hilbert-Lukasiewicz axiom 3.
fmla = '(not A imp B) imp (not A imp not B) imp A'
show_formula(fmla)

prf_str = '''
1. .hyp
proves
  2. not A imp B .hyp
  proves
    3. not A imp not B .hyp
    proves
      4. not A .hyp
      proves
      5. B .imp elim 2,4
      6. not B .imp elim 3,4
      7. bot .bot intro 5,6
    8. A .not elim 4-7
  9. (not A imp not B) imp A .imp intro 3-8
10. (not A imp B) imp (not A imp not B) imp A .imp intro 2-9
'''
proof = parse_fitch(prf_str)
proof.show_validation_result()
proof.show_fitch_text(1)


<IPython.core.display.Math object>

The proof is all valid.

│1. 	 .hyp
├─
││2. not A imp B	 .hyp
│├─
│││3. not A imp not B	 .hyp
││├─
││││4. not A	 .hyp
│││├─
││││5. B	✓ imp elim 2,4
││││6. not B	✓ imp elim 3,4
││││7. bot	✓ bot intro 5,6
│││8. A	✓ not elim 4-7
││9. (not A imp not B) imp A	✓ imp intro 3-8
│10. (not A imp B) imp (not A imp not B) imp A	✓ imp intro 2-9


In [19]:
# Exercises
# 1. Construct a Fitch proof
# 2. Verify by truth table

# A and B iff B and A
# A or B iff B or A
# A and (B and C) iff (A and B) and C
# A or (B or C) iff (A or B) or C
# A and (B or C) iff (A and B) or (A and C)
# A or (B and C) iff (A or B) and (A or C)
# A imp B, not B ├─ not A
# A or B, not A ├─ B
# not(A and B), A ├─ not B
# not A imp A ├─ A
# not A ├─ A imp B
# ├─ (B or C imp A) iff (B imp A) and (C imp A)
# ├─ (A imp B and C) iff (A imp B) and (A imp C)
# A imp B ├─ not B imp not A
# (A imp B) iff (not A or B)
# A  or B iff (not A imp B)
# not(A imp B) iff A and not B
# not(A and B) iff not A or not B
# not(A or B) iff not A and not B
# A or C, B or not C ├─ A or B
# (B and C imp A) iff (B imp A) or (C imp A) 
# (A imp B or C) iff (A imp B) or (A imp C)
# (A and not B) or (not A and B) iff not (A iff B)
# (A imp not B) and (A or B) iff not(A iff B)
# (A and B) or (not A and not B) iff (A iff B)
# (A imp not B) and (B imp not A) and (A or B) iff not(A iff B)
# (A imp B) and (not A imp C) iff (A and B) or (not A and C)
# A imp B or C, A and B imp C ├─ A imp C
# A imp B or C, A and D imp C ├─ A and (B imp D) imp C
# A or B iff (A imp B) imp B)
# ((A imp B) imp A) imp A
# A and B imp C or D ├─ B imp C or (A imp D)
# (A iff B) or (B iff C) or (C iff A)